In [13]:
!pip install flask pyngrok

In [25]:
%%writefile app.py
from flask import Flask, request, jsonify, render_template
import numpy as np
import tensorflow as tf
import json

app = Flask(__name__)

# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/driverBehaviourPrediction/driver_behavior_model.h5')

# Load scaling parameters
with open('/content/drive/MyDrive/driverBehaviourPrediction/scaling_params.json', 'r') as file:
    scaling_params = json.load(file)

mean_values = np.array(scaling_params['mean'])
std_values = np.array(scaling_params['std'])

# HTML template with all fields
html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>Driver Behavior Prediction</title>
    <style>
        body { font-family: Arial, sans-serif; text-align: center; margin: 0; padding: 0; background-color: #f5f5f5; }
        .container { margin-top: 50px; background: #fff; padding: 20px; border-radius: 10px; display: inline-block; box-shadow: 0px 0px 10px 0px rgba(0,0,0,0.1); }
        input[type="text"], button { margin: 5px 0; padding: 10px; width: 300px; font-size: 16px; border-radius: 5px; border: 1px solid #ccc; }
        button { background-color: #007bff; color: white; cursor: pointer; }
        button:hover { background-color: #0056b3; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Driver Behavior Prediction</h1>
        <form action="/predict" method="post">
            <input type="text" name="Timestamp" placeholder="Timestamp (e.g., 0.5)" required><br>
            <input type="text" name="Activation Bool" placeholder="Activation Bool (e.g., 0.0)" required><br>
            <input type="text" name="X Accel (Gs)" placeholder="X Accel (Gs) (e.g., 0.2)" required><br>
            <input type="text" name="Y Accel (Gs)" placeholder="Y Accel (Gs) (e.g., -0.1)" required><br>
            <input type="text" name="Z Accel (Gs)" placeholder="Z Accel (Gs) (e.g., 0.3)" required><br>
            <input type="text" name="X Accel KF (Gs)" placeholder="X Accel KF (Gs) (e.g., -0.4)" required><br>
            <input type="text" name="Y Accel KF (Gs)" placeholder="Y Accel KF (Gs) (e.g., 0.1)" required><br>
            <input type="text" name="Z Accel KF (Gs)" placeholder="Z Accel KF (Gs) (e.g., -0.2)" required><br>
            <input type="text" name="Roll (deg)" placeholder="Roll (deg) (e.g., 0.6)" required><br>
            <input type="text" name="Pitch (deg)" placeholder="Pitch (deg) (e.g., -0.5)" required><br>
            <input type="text" name="Yaw (deg)" placeholder="Yaw (deg) (e.g., 0.2)" required><br>
            <input type="text" name="Speed (km/h)" placeholder="Speed (km/h) (e.g., 80.0)" required><br>
            <input type="text" name="GPS Speed (km/h)" placeholder="GPS Speed (km/h) (e.g., 78.0)" required><br>
            <input type="text" name="Distance to Vehicle (m)" placeholder="Distance to Vehicle (m) (e.g., 25.0)" required><br>
            <input type="text" name="Impact Time (s)" placeholder="Impact Time (s) (e.g., 0.5)" required><br>
            <input type="text" name="Car Position (m)" placeholder="Car Position (m) (e.g., -0.2)" required><br>
            <input type="text" name="Phi" placeholder="Phi (e.g., 0.1)" required><br>
            <input type="text" name="Road Width (m)" placeholder="Road Width (m) (e.g., 3.0)" required><br>
            <input type="text" name="Lane Estimator State" placeholder="Lane Estimator State (e.g., 1.5)" required><br>
            <button type="submit">Predict</button>
        </form>
    </div>
</body>
</html>
"""

@app.route('/')
def home():
    return html_template

@app.route('/predict', methods=['POST'])
def predict():
    # Collect input data
    input_features = [
        float(request.form['Timestamp']),
        float(request.form['Activation Bool']),
        float(request.form['X Accel (Gs)']),
        float(request.form['Y Accel (Gs)']),
        float(request.form['Z Accel (Gs)']),
        float(request.form['X Accel KF (Gs)']),
        float(request.form['Y Accel KF (Gs)']),
        float(request.form['Z Accel KF (Gs)']),
        float(request.form['Roll (deg)']),
        float(request.form['Pitch (deg)']),
        float(request.form['Yaw (deg)']),
        float(request.form['Speed (km/h)']),
        float(request.form['GPS Speed (km/h)']),
        float(request.form['Distance to Vehicle (m)']),
        float(request.form['Impact Time (s)']),
        float(request.form['Car Position (m)']),
        float(request.form['Phi']),
        float(request.form['Road Width (m)']),
        float(request.form['Lane Estimator State'])
    ]

    # Normalize the input data using the scaling parameters
    input_features_normalized = (np.array(input_features) - mean_values) / std_values

    # Make a prediction
    prediction = model.predict(input_features_normalized.reshape(1, -1))
    predicted_class = np.argmax(prediction)

    # Map prediction to labels
    labels = ['NORMAL', 'AGGRESSIVE', 'DROWSY']
    result = labels[predicted_class]

    return f"<h1>Predicted Behavior: {result}</h1>"

if __name__ == '__main__':
    app.run()



Overwriting app.py


In [26]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://xt93z86eqch-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
!python app.py
from pyngrok import ngrok

# Open an HTTP tunnel to the Flask app
public_url = ngrok.connect(5000)
print("Public URL:", public_url)
# Now your app is accessible at the public URL
